# Blinky

This blinky variant simply toggles a LED when a counter unit reaches  certain value. First, we import the necessary cyHDL API:

In [1]:
import sys
sys.path.insert(0, "../..")
from cyhdl import *

We define a basic type `Bool` and a `Register` type of with 8:

In [2]:
Bool = Signal.Type(bool)
Register = Signal.Type(intbv, 10)

Finally, we define a hardware block with a few pins and a `cfg` register:

In [3]:
@block
def blinky(clk   : ClkSignal,
           en    : Bool,
           reset : ResetSignal,
           outp  : Bool.Output,
           cfg   : Register  ):

    counter = Signal(intbv(0)[16:])
    val = Bool(0)
    
    @always_seq(clk.posedge, reset)
    def worker():
        # When enable set, set counter's .next value to incremented
            
        if counter[:8] == cfg[8:]:
            val.next = ~val # remember: assign signals using .next! 
            counter.next = 0
        elif en:
            counter.next = counter + 1

            
    # This is a simple list of primitives. A .wireup is a simple assignment outside
    # a process
    logic = [
        outp.wireup(val)
            ]
    
    return instances()

## Testing/simulating the unit

In order to simulate the functionality of the blinky unit, we need to build a test bench - like virtual peripherals stimulating and probing the pins of the unit under test (`uut`).

We use an external simulator (GHDL or Icarus Verilog) in this configuration. The testbench including the uut is emit to the target HDL and passed on to the chosen simulator backend. There is no co-simulation taking place in this scenario.

Because the testbench code is transpiled to HDL, a `@block` decorator is present. Note that a testbench executed in native Python (using a different `Sim` simulator backend) would omit that decorator.

Because Signal classes can depend on the simulator, future compatible practise is to use the simulator signal class members as top level signals in the test bench. Because writing portable testbenches is of a more complex subject, we stick with the static variant below:

In [5]:
from myirl.test.ghdl import GHDL as Sim
#from myirl.test.icarus import ICARUS as Sim
from cyrite.simulation import sim

@sim.testbench(Sim, time_unit = 'ns')
@block
def testbench():
    clk = Sim.ClkSignal()
    reset = Sim.ResetSignal(1, 1, isasync = False)
    
    assert reset.init == True
    
    ce = Sim.Signal(bool())
    
    led = Sim.Signal(bool())
    
    a = Sim.Signal(intbv()[10:])

    uut = blinky(clk = clk, en = ce, reset = reset, outp = led, cfg = a)

    @always(delay(2))
    def clkgen():
        clk.next = ~ clk

    @sequence
    def main():
        yield delay(1)
        assert reset == True
        yield delay(20)
        a.next = 6
        yield delay(10)
        reset.next = False
        yield delay(10)
        assert led == False  # XXX Not on CXXRTL simulator
        ce.next = True
        yield delay(20)
        print("Wait for output to toggle")
        # This waits for a rising edge of the led signal:
        yield led.posedge # XXX non-portable to CXXRTL simulator
        print("Output toggled")
        yield delay(50)
        print("Simulation finished")
        raise StopSimulation

    return locals()

This test bench simply waits for the `led` output to become high using a `yield sig.posedge` construct.

A few important things to note:
   * A .posedge or .negedge event waiter using yield is not portable among all simulator back ends and is only guaranteed to work via a target language
   * Always terminate the simulation with a `raise StopSimulation` statement, otherwise simulation may keep running in some simulation back ends
   * The @block decorator is necessary when a translation of the testbench to another language occurs. This is not the case with a co-simulation setup, where the testbench is not translated to another HDL, like the CXXRTL simulator backend
   
Finally, we instance the testbench and run it:

In [6]:
def test_simulation(n):
    t = testbench()
    t.run(n, debug = True, wavetrace = "testbench.vcd")
    return t

t = test_simulation(20000)

../../myirl/kernel/_types.py:542: UserWarning: Non-ClkSignal 'led' edge event reference.
You might want to use ClkSignal instead.
  base.warnings.warn("Non-ClkSignal '%s'" % obj.identifier + \


 Writing 'blinky' to file /tmp/blinky.vhdl 
 Writing 'testbench' to file /tmp/testbench.vhdl 
 Creating library file /tmp/module_defs.vhdl 
DEBUG_FILES ['/tmp/blinky.vhdl', '/tmp/testbench.vhdl', '/tmp/module_defs.vhdl']
==== COSIM stdout ====
analyze /home/testing/src/myhdl2/myirl/targets/../test/vhdl/txt_util.vhdl
analyze /home/testing/src/myhdl2/myirl/targets/libmyirl.vhdl
analyze /tmp/blinky.vhdl
analyze /tmp/testbench.vhdl
elaborate testbench

==== COSIM stdout ====
Wait for output to toggle
Output toggled
Simulation finished
simulation stopped @6236ns



In [7]:
!cat /tmp/testbench.vhdl

-- File generated from source:
--     ../../myirl/emulation/myhdl.py
-- (c) 2016-2022 section5.ch
-- Modifications may be lost, edit the source file instead.

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

library work;

use work.txt_util.all;
use work.myirl_conversion.all;

entity testbench is
end entity testbench;

architecture myhdl_emulation of testbench is
    -- Local type declarations
    -- Signal declarations
    signal led : std_ulogic;
    signal clk_c209 : std_ulogic := '0';
    signal ce : std_ulogic;
    signal reset : std_ulogic := '1';
    signal a : unsigned(9 downto 0);
begin
    
    -- Instance blinky
    inst_blinky_0: entity work.blinky
    port map (
        clk => clk_c209,
        en => ce,
        reset => reset,
        outp => led,
        cfg => a
    );
    
clkgen:
    clk_c209 <= not clk_c209 after 2.000000 ns;
    
    
main:
    process
    begin
        wait for 1 ns;
        assert (reset = '1')
            report "Failed in /t

## Advanced simulation

The above check for a rising signal is a little cheap. We would like to verify a real blink timing.
Moreover, there are non-HDL simulation back ends that do not transpile the code as noted. To easily switch between various simulation and synthesis targets, a top level design module class is the better investment into the future of possibly growing complexity of the design.

In particular, when designs get big, simulation can slow down. A way to speed this up is using a less timing precise but faster running edition of compiled logic using the CXXRTL backend of yosys.

Due to its architecture and lack of built-in delta stepping capabilities, a statement such as the above `yield led.posedge` is not allowed in this context.

See [Simulation](simulation.ipynb) on how complex simulation designs are set up.